In [8]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [9]:
import pandas as pd
data= pd.read_csv('sentences.csv')
data.head()

,ID,Sentence
0,1,The cat is sleeping on the couch.
1,2,The dog is resting on the sofa.
2,3,The kitten is napping on the chair.
3,4,I love eating pizza for dinner.
4,5,I enjoy having pasta for lunch.


In [10]:
def preprocessing_pipeline(text):
    text=text.lower()
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F" 
        "\U0001F300-\U0001F5FF" 
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f" 
        "\u3030"
        "]+", re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text
data['preprocessed_text'] = data['Sentence'].apply(preprocessing_pipeline)
data.head()

,ID,Sentence,preprocessed_text
0,1,The cat is sleeping on the couch.,cat sleeping couch
1,2,The dog is resting on the sofa.,dog resting sofa
2,3,The kitten is napping on the chair.,kitten napping chair
3,4,I love eating pizza for dinner.,love eating pizza dinner
4,5,I enjoy having pasta for lunch.,enjoy pasta lunch


In [11]:
def bag_of_words(corpus):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    try:
        feature_names = vectorizer.get_feature_names_out()
    except AttributeError:
        feature_names = vectorizer.get_feature_names()
    bow_df = pd.DataFrame(X.toarray(), columns=feature_names)
    return bow_df

In [12]:
bag_of_words_df = bag_of_words(data['preprocessed_text'])
print(bag_of_words_df.head(10))

   ability  acquiring  affect  afternoon  beach  bed  book  break  bright  \
0        0          0       0          0      0    0     0      0       0   
1        0          0       0          0      0    0     0      0       0   
2        0          0       0          0      0    0     0      0       0   
3        0          0       0          0      0    0     0      0       0   
4        0          0       0          0      0    0     0      0       0   
5        0          0       0          0      0    0     0      0       0   
6        0          0       0          0      0    0     0      0       0   
7        0          0       0          1      0    0     0      0       1   
8        0          0       0          0      0    0     0      0       0   
9        0          0       0          0      0    1     1      0       0   

   burger  ...  temperature  tide  time  today  upon  warm  water  wave  \
0       0  ...            0     0     0      0     0     0      0     0   
1 

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(bag_of_words_df)
similarity_df = pd.DataFrame(similarity_matrix, index=data['Sentence'], columns=data['Sentence'])
print(similarity_df)

Sentence                                            The cat is sleeping on the couch.  \
Sentence                                                                                
The cat is sleeping on the couch.                                            1.000000   
The dog is resting on the sofa.                                              0.000000   
The kitten is napping on the chair.                                          0.000000   
I love eating pizza for dinner.                                              0.000000   
I enjoy having pasta for lunch.                                              0.000000   
I like consuming burgers for supper.                                         0.000000   
The weather today is sunny and warm.                                         0.000000   
The climate this afternoon is bright and hot.                                0.000000   
The temperature right now is clear and pleasant.                             0.000000   
She reads books every

In [14]:
from sklearn.metrics import jaccard_score
import numpy as np
binary_bow_df = (bag_of_words_df > 0).astype(int)
jaccard_matrix = np.zeros((binary_bow_df.shape[0], binary_bow_df.shape[0]))
for i in range(binary_bow_df.shape[0]):
    for j in range(binary_bow_df.shape[0]):
        vec_i=binary_bow_df.iloc[i].values
        vec_j=binary_bow_df.iloc[j].values
        intersection=np.sum(np.logical_and(vec_i,vec_j))
        union=np.sum(np.logical_or(vec_i,vec_j))
        if union==0:
            jaccard_matrix[i][j]=0.0
        else:
            jaccard_matrix[i][j]=intersection/union
jaccard_df=pd.DataFrame(jaccard_matrix,index=data['Sentence'],columns=data['Sentence'])
print(jaccard_df)

Sentence                                            The cat is sleeping on the couch.  \
Sentence                                                                                
The cat is sleeping on the couch.                                            1.000000   
The dog is resting on the sofa.                                              0.000000   
The kitten is napping on the chair.                                          0.000000   
I love eating pizza for dinner.                                              0.000000   
I enjoy having pasta for lunch.                                              0.000000   
I like consuming burgers for supper.                                         0.000000   
The weather today is sunny and warm.                                         0.000000   
The climate this afternoon is bright and hot.                                0.000000   
The temperature right now is clear and pleasant.                             0.000000   
She reads books every